In [118]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import utils, datasets, transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F

manualSeed = 999
print("Random Seed: ", manualSeed)
torch.manual_seed(manualSeed)
torch.use_deterministic_algorithms(True)

Random Seed:  999


In [119]:
torch.manual_seed(0)

In [120]:
dataroot = "/home/maxim/Documents/TestProject/gans/Face/img_align_celeba"

workers = 20

batch_size = 128

image_size = 64

In [121]:
dataset = datasets.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
print(f'Total Size of Dataset: {len(dataset)}')

Total Size of Dataset: 202599


In [122]:
dataloader = DataLoader (
    dataset=dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=workers
)

In [125]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [126]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d( 100, 64 * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(64 * 8),
            nn.ReLU(True),

            nn.ConvTranspose2d(64 * 8, 64 * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 4),
            nn.ReLU(True),

            nn.ConvTranspose2d( 64 * 4, 64 * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 2),
            nn.ReLU(True),

            nn.ConvTranspose2d( 64 * 2, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            
            nn.ConvTranspose2d( 64, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)

In [127]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 64 * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64 * 2, 64 * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 4),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(64 * 4, 64 * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64 * 8),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(64 * 8, 1, 4, 1, 0, bias=False)
        )

    def forward(self, input):
        return self.main(input)

In [128]:
import pytorch_lightning as pl
import torchvision.utils as vutils

In [129]:
criterion = nn.BCEWithLogitsLoss()

class GAN(pl.LightningModule):
    def __init__(self):
        super().__init__()

        self.gen = Generator().to("cuda:0")
        self.gen.apply(weights_init)
        self.disc = Discriminator().to("cuda:0")
        self.disc.apply(weights_init)

        self.automatic_optimization = False

    def forward(self, noise):
        return self.gen(noise)
    
    def training_step(self, batch, batch_idx):
        X, y = batch

        # batch[0].to("cuda:0")

        g_optimizer, d_optimizer = self.optimizers()
	
        self.disc.zero_grad()
        
        real_image = batch[0].to("cuda:0")
        b_size = real_image.size(0)

        real_label = torch.full((b_size,), 1., device="cuda:0")
        fake_label = torch.full((b_size,), 0., device="cuda:0")

        output = self.disc(real_image).view(-1).to("cuda:0")
        errD_real = criterion(output, real_label)
        errD_real.backward()
        D_x = output.mean().item()

        noise = torch.randn(b_size, 100, 1, 1, device="cuda:0")
        # noise = torch.randn(b_size, 192, 1, 1, device="cuda:0")
        fake = self.gen(noise)
        output = self.disc(fake.detach()).view(-1).to("cuda:0")
        errD_fake = criterion(output, fake_label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        d_optimizer.step()

        self.gen.zero_grad()
        output = self.disc(fake).view(-1).to("cuda:0")
        errG = criterion(output, real_label)
        errG.backward()
        D_G_z2 = output.mean().item()
        g_optimizer.step()

        print(
			f'Loss-D: {errD.item():.4f}',
			f'Loss-G: {errG.item():.4f}',
			f'D(x): {D_x:.4f}',
			f'D(G(z)): [{D_G_z1:.4f}/{D_G_z2:.4f}]',
			end='\r'
		)

        self.log("Loss Discriminator", errD.item())
        self.log("Loss Generator", errG.item())

        self.gen.eval()

        noise = torch.randn(b_size, 100, 1, 1, device="cuda:0")
        # noise = torch.randn(b_size, 192, 1, 1, device="cuda:0")
        with torch.no_grad():
            fake = self.gen(noise)

        grid = vutils.make_grid(fake)
        self.logger.experiment.add_image("generated_images", grid, self.current_epoch)

        return errG
        
    def configure_optimizers(self):
        opt_g = torch.optim.Adam(self.gen.parameters(), lr = 0.0002, betas=(0.5, 0.999))
        opt_d = torch.optim.Adam(self.disc.parameters(), lr = 0.0002, betas=(0.5, 0.999))

        return [opt_g, opt_d], []

In [130]:
model = GAN()

trainer = pl.Trainer(accelerator="gpu", devices=1, max_epochs=1, precision="32")

trainer.fit(model, train_dataloaders=dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type          | Params
---------------------------------------
0 | gen  | Generator     | 3.6 M 
1 | disc | Discriminator | 2.8 M 
---------------------------------------
6.3 M     Trainable params
0         Non-trainable params
6.3 M     Total params
25.369    Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


In [131]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

Reusing TensorBoard on port 6006 (pid 468298), started 1:03:39 ago. (Use '!kill 468298' to kill it.)